# 0. Общий код

Ячейка подключает библиотеки и задаёт функции, используемые пунктах 1.1. и далее для сокращения объёма кода.

In [ ]:
# 0. Общий код

from os import walk
import xml.etree.ElementTree as ET
from xml.dom import minidom
import re
import codecs
from transliterate import translit, get_available_language_codes
from random import randint


############################ VARS ################################
# Symbols and combinations that have to be corrected after OCR
COMBINATIONS_CORR_GLOBAL = {
	'A':'А', 'a':'а', 'B':'В', 'b':'Ь', 'C':'С', 'c':'с', 'E':'Е', 'e':'е', 'H':'Н', 'K':'К', 'M':'М', 'O':'О', 'P':'Р', 'p':'р', 'T':'Т', 'X':'Х', 'y':'у',
	#'x':'х', # Causes infinite loops for some reason
	'U' : 'И',
	'r' : 'г',
	'n' : 'п',
	'Y' : 'У',
	' -' : '-',
	'- ' : '-',
	'S' : 'Я',
	'0' : 'О',
	'3' : 'З',
	'6' : 'б',
	'І' : 'I'		# This teo are different!
}
# Symbols excluded in xml have to be converted back
XML_EXCLUDES = {
	'&quot;' : '"',
	'&apos;' : "'",
	'&lt;' : '<',
	'&gt;' : '>',
	'&amp;' : '&'
}
##################################################################


# Write xml tree to file
def prettify(elem):
	# Pretty-printed XML string for the Element.
	rough_string = ET.tostring(elem, 'utf-8')
	reparsed = minidom.parseString(rough_string)
	return reparsed.toprettyxml(indent="  ")
def xml_write(root, filename):
	with codecs.open(filename, 'w', 'utf-8') as f:
		f.write(prettify(root))


# Convert xml excluded symbols
def xml_excluded_convert (text:str) -> str:
	for key in XML_EXCLUDES.keys():
		while text.find(key) != -1:
			pos = text.find(key)
			text = text[:pos] + XML_EXCLUDES[key] + text[pos+len(key):]
	return text


def remove_xml_spaces(elem, filename):
	elem.tail = None
	if elem.text != None:
		is_space = True
		for letter in elem.text:
			is_space = False if letter != ' ' else is_space
		if is_space:
			elem.text = None
		else:
			if elem.tag == 'text':
				elem.text = get_texts(filename)[0]
			elif elem.tag == 'text_orig':
				elem.text = get_texts(filename)[1]
			elem.text = xml_excluded_convert(elem.text)
	for subelem in elem:
		subelem = remove_xml_spaces(subelem, filename)
	return elem
def parse_xml(filename):
	# Parse existing xml (string parsing is needed to avoid extra newlines appearing)
	exit_string = ''
	with codecs.open(filename, 'r', 'utf-8') as f:
		for i in f.readlines():
			exit_string += i[:-1]
	root = ET.fromstring(exit_string)
	root = remove_xml_spaces(root, filename)
	return root


# !!!BUG!!! for some reason newlines diappear in texts in parsed xml, so extract article texts manually and replace
def get_texts(filename:str) -> str:
	with codecs.open(filename, 'r', 'utf-8') as f:
		file = f.read()
	text = file[file.find('<text>')+6:file.find('</text>')]
	with codecs.open(filename, 'r', 'utf-8') as f:
		file = f.read()
	text_orig = file[file.find('<text_orig>')+11:file.find('</text_orig>')]
	return (text, text_orig)


# Get xml tree element wit sertain tag name
def get_xml_elem(root:ET.Element, elem_path:str) -> ET.Element:
	tgt = elem_path.split('/')[0]
	for elem in root:
		if elem.tag == tgt:
			if elem_path.find('/') != -1:
				return get_xml_elem(elem, elem_path[elem_path.find('/')+1:])
			else:
				return elem
	return None


# Small dictionaries merger
def dict_merge(dict1:dict, dict2:dict) -> dict:
	for key in dict2.keys():
		dict1[key] = dict2[key]
	return dict1

# 1. Базовый парсер заголовков

Вытаскивает из latex-кода заголовки статей и их расположение в файлах.

Разбивка происходит в полуручном режиме, т.к. нет уверенности в формате заголовков.

В тексте ищутся слова, содержащие в своём составе заглавные буквы на русском и английском языках в отношении, большем или равным заданному (по умолчанию 0.51, при меньших значениях количество вхождений значительно возрастает, например за счёт двухбуквенных предлогов). Предполагается, что таким образом удаётся обнаруживать неправильно машиинно распознанный капс. Слова или цепочки слов, состоящие из одного строчного символа включаются в заголовок, если стоят между слов, определённых как часть заголовка. При этом, одиночные заглавные буквы, а также инициалы не воспринимаются как начало заголовка.

## Использование
- При удовлетворительном определении заголовка нажать `Enter` без дополнительного ввода.
- Если предложенное место заголовком не является ввести `"n"`
- При неправильном определении границ заголовка ввести два корректировочных числа для сдвига левой и правой границы.
  - ЗАМЕЧАНИЕ: сдвиг производится попробельно, т.е. двойной пробел будет распознан как слово нулевой длины.
  - ЗАМЕЧАНИЕ: границы отображаемого фрагмента текста будут передвинуты автоматически. Длины левой и правой границ в словах задаются в параметрах.
  - ПРИМЕРЫ:
    - `out: a [B C] d e f` -> `in: 0 2` -> `out: a [B C D E] f`
    - `out: a b c [D E] f` -> `in: 2 -1` -> `out: a [B C D] e f`
- Также возможен посимвольный сдвиг правой границы в случае "сращивания" заголовка статьи и её текста. Ввести одно число, начиная с точки.
  - ПРИМЕРЫ:
    - `out: a[BC]def` -> `in: .2` -> `out: a[BCDE]f`
    - `out: a[BCDE]f` -> `in: .-1` -> `out: a[BCD]ef`

В выводе в терминале переносы строк для удобства заменены на `"$"`

### Прочее
- Для определителя капса достуны исключения, которые никогда не будут рассматриваться, как потенциальные начала заголовков, см. опции. По умолчанию: первые 10 римских цифр, "МэВ" и "ГэВ".
- Использовать системный терминал для взаимодействия оказывается удобнее, чем использовать jupyter, поэтому можно скопировать ячейку с кодом в файл `scripter.py` и запускать его.
- При положительном определении заголовка файл дополняется немедленно, прервать процесс можно в любой момент, как и продолжить после -- итоговый файл будет дополяться, а не перезаписываться с нуля при новом запуске программы (главное не забыть предварительно удалить из конца файла дубликаты, если вы начинаете с той страницы, на которой закончили в прошлый раз, а не со следующей).

In [ ]:
# 1. Базовый парсер заголовков

from os import walk
import xml.etree.ElementTree as ET
from xml.dom import minidom
import re
import codecs


############################ VARS ################################
PAGES_DIR = "./matphys/rpages/"
EXIT_DIR = "./matphys/"
EXIT_FILE = "FMEv2.xml"
# First and last pages to be parsed
START_PAGE = 372
END_PAGE = 400
# How many words to display before and after a potential title
LEAD_WORDS = 5
AFT_WORDS = 5
# Look in the description
CAPS_QUOT = 0.51
EXCEPTIONS = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'МэВ', 'ГэВ']
# Symbols excluded in xml have to be converted back
XML_EXCLUDES = {'&quot;' : '"', '&apos;' : "'", '&lt;' : '<',	'&gt;' : '>',	'&amp;' : '&'}
##################################################################



class Article:
	start_title = 0
	end_title = 0
	filename = ''



# Write xml tree to file
def prettify_1(elem):
	# Pretty-printed XML string for the Element.
	rough_string = ET.tostring(elem, 'utf-8')
	reparsed = minidom.parseString(rough_string)
	return reparsed.toprettyxml(indent="  ")
def xml_write_1(root):
	with codecs.open(EXIT_DIR + EXIT_FILE, 'w', 'utf-8') as f:
		f.write(prettify_1(root))


# Get filenames needed
filenames_raw = next(walk(PAGES_DIR), (None, None, []))[2]  # [] if no file
filenames = []
for i in range(START_PAGE, END_PAGE + 1):
	for filename in filenames_raw:
		beginning = "rp-" + str(i) + "_"
		if filename[:len(beginning)] == beginning and filename[-4:] == ".mmd":
			filenames.append(filename)
						

# Check for existing xml
filenames_raw = next(walk(EXIT_DIR), (None, None, []))[2]  # [] if no file
if not(EXIT_FILE in filenames_raw):
	root = ET.Element('data')
	xml_write_1(root)


# Convert xml excluded symbols
def xml_excluded_convert (text:str) -> str:
	for key in XML_EXCLUDES.keys():
		while text.find(key) != -1:
			pos = text.find(key)
			text = text[:pos] + XML_EXCLUDES[key] + text[pos+len(key):]
	return text
def remove_xml_spaces_1(elem):
	elem.tail = None
	if elem.text != None:
		is_space = True
		for letter in elem.text:
			is_space = False if letter != ' ' else is_space
		elem.text = None if is_space else xml_excluded_convert(elem.text)
	for subelem in elem:
		subelem = remove_xml_spaces_1(subelem)
	return elem
def parse_xml_1():
	# Parse existing xml (string parsing is needed to avoid extra newlines appearing)
	exit_string = ''
	with codecs.open(EXIT_DIR + EXIT_FILE, 'r', 'utf-8') as f:
		for i in f.readlines():
			exit_string += i[:-1]
	root = ET.fromstring(exit_string)
	# Remove empty tails and texts
	root = remove_xml_spaces_1(root)
	return root
root = parse_xml_1()
num = len(root) + 1


# Add article title and metadata to xml tree
def add_artice_1(elem):
	# Update root in case it's been changed
	root = parse_xml_1()
	num = len(root) + 1
	article = ET.SubElement(root, 'article', {'n':str(num)})
	title = ET.SubElement(article, 'title')
	title.text = file[elem.start_title+1:elem.end_title]
	title_meta = ET.SubElement(article, 'title-meta')
	title_file = ET.SubElement(title_meta, 'title-file')
	title_file.text = elem.filename
	title_start = ET.SubElement(title_meta, 'title-start')
	title_start.text = str(elem.start_title + 1)
	title_end = ET.SubElement(title_meta, 'title-end')
	title_end.text = str(elem.end_title)
	xml_write_1(root)
	return num


# Count number of alphabetic letters in word
def count_letters_1(word):
	num = 0
	for letter in word:
		num += 0 if re.match(r"[A-ZА-Яa-zа-я]", letter) == None else 1
	return num

# Check if word is written in CAPS
def check_caps_1(word):
	num = 0
	len_word = 0
	for letter in word:
		#num += 0 if re.match(r"[A-ZА-Я0-9]|[!#$%&'*+-.^_`|~:]", letter) == None else 1					# Too many symbols, math formulas are being detected
		len_word += 1 if re.match(r"[!#$%&'*+-.^_`|~:]", letter) == None else 0
		num += 0 if re.match(r"[A-ZА-Я]", letter) == None else 1
	return 0 if len_word == 0 or num / len_word < CAPS_QUOT or word in EXCEPTIONS else num				# Also exclude common roman numbers

# Check for initials like "I.E."
def check_initials_1(word):
	initials = True
	for i in range(len(word) - 1):
		type_1 = 0 if re.match(r"[A-ZА-Яa-zа-я]", word[i]) == None else 1
		type_2 = 0 if re.match(r"[A-ZА-Яa-zа-я]", word[i + 1]) == None else 1
		initials = False if type_1 and type_2 else initials
	return initials


# Find next ot prev word boundary (space / newline)
def prev_from_1(pos, file):
	pos = max(pos, 0)
	prev_space = file.rfind(' ', 0, pos)
	prev_nl = file.rfind('\n', 0, pos)
	prev_space = -1 if prev_space == -1 else prev_space
	prev_nl = -1 if prev_nl == -1 else prev_nl
	return max(prev_nl, prev_space)
def next_from_1(pos, file, end_replace = True):
	next_space = file.find(' ', pos + 1)
	next_nl = file.find('\n', pos + 1)
	if end_replace:
		next_space = len(file) if next_space == -1 else next_space
		next_nl = len(file) if next_nl == -1 else next_nl
	return max(next_nl, next_space) if next_space == -1 or next_nl == -1 else min(next_nl, next_space)


# Main loop
for filename in filenames:
	print()
	print("################################ " + filename + " ################################")
	with codecs.open(PAGES_DIR + filename, 'r', 'utf-8') as f:
		file = f.read()
	
	word_bound_l = -1
	word_bound_r = next_from_1(word_bound_l, file, end_replace=False)
	EOF_reached = False

	while not EOF_reached:
		if word_bound_r == -1:
			word_bound_r = len(file)
			EOF_reached = True


		if check_caps_1(file[word_bound_l+1:word_bound_r]) < 2 or check_initials_1(file[word_bound_l+1:word_bound_r]):
			word_bound_l = word_bound_r
			word_bound_r = next_from_1(word_bound_l, file, end_replace=False)
		
		else: # Possibly found a title
			# Left border of a title is already known
			start_title = word_bound_l

			# Define right border of a title
			defined_end = False
			end_title = word_bound_r
			while not defined_end:
				word_bound_l = word_bound_r
				word_bound_r = next_from_1(word_bound_l, file)

				if word_bound_l == len(file):
					defined_end = True
				elif not check_caps_1(file[word_bound_l+1:word_bound_r]) and count_letters_1(file[word_bound_l+1:word_bound_r]) < 2:
					if re.match(r"[A-ZА-Яa-zа-я]", file[word_bound_l+1]) != None:
						# Most possibly belongs to title
						end_title = word_bound_r
					else:
						# Most possibly NOT belongs to title
						pass
				elif check_caps_1(file[word_bound_l+1:word_bound_r]):
					end_title = word_bound_r
				else:
					defined_end = True

			next_title = False
			while not next_title:
				# Update root in case it's been changed
				root = parse_xml_1()
				num = len(root) + 1

				# Console output for further user actions
				segment_start = start_title
				segment_end = end_title
				for i in range(LEAD_WORDS):
					segment_start = prev_from_1(segment_start, file)
				for i in range(AFT_WORDS):
					segment_end = next_from_1(segment_end, file)
				
				out_str = file[segment_start+1:segment_end]

				# Format
				for i in range(len(out_str)):
					out_str = out_str[:i] + ('$' if out_str[i] == '\n' else out_str[i]) + out_str[i+1:]
				out_str = f"{num})\n" + out_str + '\n' + ' ' * (start_title - segment_start) + '^' * (end_title - start_title - 1)
				# Check for "section" in the string. This is referred to alphabetic tip at the bottom of the page
				"""if 'section' in out_str or 'title' in out_str:
					out_str += '     ############################### Title or section found! ###############################'""" # Not Used
				print(out_str)

				# User actions
				response = input()
				try:
					if response == '':
						# Add article
						article = Article()
						article.start_title = start_title
						article.end_title = end_title
						article.filename = filename
						num = add_artice_1(article)
						next_title = True
						word_bound_l = end_title
						word_bound_r = next_from_1(word_bound_l, file, end_replace=False)
						print(f'Adding article, n="{num}", title="{file[start_title+1:end_title]}"\n\n')
					elif response == 'n' or response == 'т':
						# Do not add this one
						next_title = True
						print("Not an article, skipping\n\n")
					elif response[0] == '.':
						end_title += int(response[1:])
						print("Changing title right border\n\n")
					else:
						# Change title borders
						corrections = response.split(' ')
						corrections[0] = int(corrections[0])
						corrections[1] = int(corrections[1])
						if corrections[0] > 0:
							for i in range(abs(corrections[0])):
								start_title = prev_from_1(start_title, file)
						if corrections[0] < 0:
							for i in range(abs(corrections[0])):
								start_title = next_from_1(start_title, file)
						if corrections[1] < 0:
							for i in range(abs(corrections[1])):
								end_title = prev_from_1(end_title, file)
						if corrections[1] > 0:
							for i in range(abs(corrections[1])):
								end_title = next_from_1(end_title, file)
						print("Changing title borders\n\n")
				except:
					print("########## !!! Failed on input, try again !!! ##########\n\n")


# End reached
print('###########################################################################################')
print('Last requested page processed. Press "Enter" to close this window.')
response = input()

# 1.1. Добавление заголовков по одному

В разделе параметров указать номер страницы и ТОЧНУЮ формулировку заголовка из сырого текста, после чего запустить ячейку

In [ ]:
# 1.1. Добавление заголовков по одному

############################ VARS ################################
PAGES_DIR = "./matphys/rpages/"
EXIT_DIR = "./matphys/"
EXIT_FILE = "FMEv2-add.xml"
# First and last pages to be parsed
PAGE = 371
TITLE = 'МУльтиполь с о с т о я н и я'
##################################################################



class Article:
	start_title = 0
	end_title = 0
	filename = ''


# Get filenames needed
filenames_raw = next(walk(PAGES_DIR), (None, None, []))[2]  # [] if no file
filenames = []
for i in range(PAGE, PAGE + 1):
	for filename in filenames_raw:
		beginning = "rp-" + str(i) + "_"
		if filename[:len(beginning)] == beginning and filename[-4:] == ".mmd":
			filenames.append(filename)
						

# Check for existing xml
filenames_raw = next(walk(EXIT_DIR), (None, None, []))[2]  # [] if no file
if not(EXIT_FILE in filenames_raw):
	root = ET.Element('data')
	xml_write(root, EXIT_DIR + EXIT_FILE)


root = parse_xml(EXIT_DIR + EXIT_FILE)


# Add article title and metadata to xml tree
def add_artice(elem, root, num):
	article = ET.SubElement(root, 'article', {'n':str(num)})
	title = ET.SubElement(article, 'title')
	title.text = file[elem.start_title+1:elem.end_title]
	title_meta = ET.SubElement(article, 'title-meta')
	title_file = ET.SubElement(title_meta, 'title-file')
	title_file.text = elem.filename
	title_start = ET.SubElement(title_meta, 'title-start')
	title_start.text = str(elem.start_title + 1)
	title_end = ET.SubElement(title_meta, 'title-end')
	title_end.text = str(elem.end_title)
	xml_write(root, EXIT_DIR + EXIT_FILE)

# Read requested file
with codecs.open(PAGES_DIR + filenames[0], 'r', 'utf-8') as f:
	file = f.read()

# Find titles and add them
start_title = 0
end_title = 0
num = len(root) + 1
while file.find(TITLE, end_title) != -1:
	start_title = file.find(TITLE, start_title)
	end_title = start_title + len(TITLE)
	start_title -= 1 # Set on space befor the title

	article = Article()
	article.start_title = max(start_title, 0)
	article.end_title = min(end_title, len(file))
	article.filename = filenames[0]
	add_artice(article, root, num)

# 2. Исправление ошибок в заголовках

Состоит из двух частей: "составитель пар" и "подстановщик".

Сначала "составитель" формирует xml-список всех заголовков с возможными автоматическими исправлениями (в формате было / стало):
1. замена латиницы на агалогичную кириллицу;
2. замена задаванных буквосочетаний (см. параметры)
3. удаление обрамляющих знаков препинания;
4. замена всех букв на заглавные (в том числе это избавляет дальнейшей необходимости исправлять имена);
5. слияние разорванных на отдельные буквы слов (если рядом оказываются несколько таких слов, то они оказываются слиты вместе).

Этот список необходимо просмотреть и исправить оставшиеся ошибки.

Затем запустить "подстановщик", который заменит все заголовки на исправленные.

## 2.1. Составитель пар "оригинальный - исправленный" для заголовков:

In [ ]:
# 2.1. Составитель пар "оригинальный - исправленный" для заголовков:

############################ VARS ################################
WORK_DIR = "./matphys/"
INPUT_FILE = "FMEv2.xml"
CORRECTION_FILE = "FMEcorr.xml"
COMBINATIONS_CORR = dict_merge(COMBINATIONS_CORR_GLOBAL, {
	'ХК' : 'Ж',
	'ЛАГРАНХ' : 'ЛАГРАНЖ',
	'ЛАТРАНХ' : 'ЛАГРАНЖ'
})
##################################################################
						

# Check for existing xml
filenames_raw = next(walk(WORK_DIR), (None, None, []))[2]  # [] if no file
if not(INPUT_FILE in filenames_raw):
	root = ET.Element('data')
	xml_write(root, WORK_DIR + CORRECTION_FILE)


root = parse_xml(WORK_DIR + INPUT_FILE)


# Get all the titles into a dict
titles_dict = {}
pages_dict = {}
for article in root:
	title = get_xml_elem(article, 'title').text
	titles_dict[title] = title
	title_file = get_xml_elem(article, 'title-meta/title-file')
	pages_dict[title] = title_file.text[title_file.text.find('-')+1:title_file.text.find('_')]


# Correct preferred combinations and latin letters
for title in titles_dict.keys():
	#print(title)
	title_new = titles_dict[title]
	for comb in COMBINATIONS_CORR.keys():
		#print(comb)
		while title_new.find(comb) != -1:
			#print(title_new.find(comb))
			title_new = title_new[:title_new.find(comb)] + COMBINATIONS_CORR[comb] + title_new[title_new.find(comb) + len(comb):]
	titles_dict[title] = title_new

# Remove bounding symbols
for title in titles_dict.keys():
	title_new = titles_dict[title]
	while re.match(r"[!#%&'*+-.^_`|~:;]", title_new[0]) != None:
		title_new = title_new[1:]
	while re.match(r"[!#%&'*+-.^_`|~:;]", title_new[-1]) != None:
		title_new = title_new[:-1]
	titles_dict[title] = title_new

# CAPS
for title in titles_dict.keys():
	titles_dict[title] = titles_dict[title].upper()

# Merge single-lettered words
for title in titles_dict.keys():
	title_new = titles_dict[title]
	title_new = ' ' + title_new + ' '
	for i in range(len(title_new) - 4):
		if (title_new[i] == ' ' or title_new[i] == '№') and title_new[i + 2] == ' ' and title_new[i + 4] == ' ':
			title_new = title_new[:i+2] + '№' + title_new[i+3:]
	i = 0
	while i < len(title_new):
		if title_new[i] == '№':
			title_new = title_new[:i] + title_new[i+1:]
			i = 0
		else:
			i += 1
	while title_new[0] == ' ':
		title_new = title_new[1:]
	while title_new[-1] == ' ':
		title_new = title_new[:-1]
	titles_dict[title] = title_new


# Write corrections xml
root = ET.Element('data')
for i in titles_dict.items():
	pair = ET.SubElement(root, 'pair')
	title_old = ET.SubElement(pair, 'title_old')
	title_old.text = i[0]
	title_new = ET.SubElement(pair, 'title_new')
	title_new.text = i[1]
	page = ET.SubElement(pair, 'page')
	page.text = pages_dict[i[0]]
xml_write(root, WORK_DIR + CORRECTION_FILE)

## 2.2. Подстановщик исправленных заголовков:

In [ ]:
# 2.2. Подстановщик исправленных заголовков:

############################ VARS ################################
WORK_DIR = "./matphys/"
INPUT_FILE = "FMEv2.xml"
CORRECTION_FILE = "FMEcorr.xml"
EXIT_FILE = "FMEtitles.xml"
##################################################################



root = parse_xml(WORK_DIR + CORRECTION_FILE)


# Get all the corrections into a dict
titles_dict = {}
for pair in root:
	titles_dict[get_xml_elem(pair, 'title_old').text] = get_xml_elem(pair, 'title_new').text


root = parse_xml(WORK_DIR + INPUT_FILE)


# Replace titles
for article in root:
	get_xml_elem(article, 'title').text = titles_dict[get_xml_elem(article, 'title').text]
xml_write(root, WORK_DIR + EXIT_FILE)

# 3. Сортировщик / сливщик файлов с заголовками

Сортирует статьи в файлах из данного списка в порядке страница-расположение, т.е. (если не сказано иного) в алфавитном порядке и выводит в один выходной файл. Также порядковый номер заменяется uri формата "http://libmeta.ru/fme/article/1_Kraevaya"

In [ ]:
# 3. Сортировщик / сливщик файлов с заголовками

############################ VARS ################################
WORK_DIR = "./results/"
INPUT_FILES = ["FMEtitles-p5-100.xml", "FMEtitles-p101-200.xml", "FMEtitles-p201-300.xml", "FMEtitles-p301-400.xml", "FMEtitles-p301-400-add.xml"]
EXIT_FILE = "FMEtitles-merged.xml"
##################################################################



class Article:
	title = ''
	start_title = ''
	end_title = ''
	filename = ''



# Add article title and metadata to xml tree
def add_artice(elem, root, num):
	article = ET.SubElement(root, 'article', {'uri':"http://libmeta.ru/fme/article/"+str(num)+"_"+translit(elem.title[:elem.title.find(' ')], 'ru', True)})
	title = ET.SubElement(article, 'title')
	title.text = elem.title
	title_meta = ET.SubElement(article, 'title-meta')
	title_file = ET.SubElement(title_meta, 'title-file')
	title_file.text = elem.filename
	title_start = ET.SubElement(title_meta, 'title-start')
	title_start.text = str(int(elem.start_title) + 1)
	title_end = ET.SubElement(title_meta, 'title-end')
	title_end.text = elem.end_title


# Collect all the articles
articles_dict = {}
for filename in INPUT_FILES:
	root = parse_xml(WORK_DIR + filename)
	for article in root:
		num = ()
		title = get_xml_elem(article, 'title').text
		elem = get_xml_elem(article, 'title-meta/title-file')
		page = elem.text[elem.text.find('-')+1:elem.text.find('_')]
		pos = get_xml_elem(article, 'title-meta/title-start').text
		start = get_xml_elem(article, 'title-meta/title-start').text
		end = get_xml_elem(article, 'title-meta/title-end').text
		file = get_xml_elem(article, 'title-meta/title-file').text
		num = (int(page), int(pos))
		articles_dict[num] = {'title':title, 'file':file, 'start':start, 'end':end}


# Sort keys and wrtite articles accordingly
root = ET.Element('data')
nums_list = sorted(list(i for i in articles_dict.keys()))
for num in range(len(nums_list)):
	article = Article()
	article.title = articles_dict[nums_list[num]]['title']
	article.start_title = articles_dict[nums_list[num]]['start']
	article.end_title = articles_dict[nums_list[num]]['end']
	article.filename = articles_dict[nums_list[num]]['file']
	add_artice(article, root, num + 1)
xml_write(root, WORK_DIR + EXIT_FILE)

# 4. Парсер текстов статей

По информации из указанного файла с заголовками вытаскивает в сыром виде тексты статей. Каждая статья помещается в свой .xml файл, с заголовком, содержащим номер статьи и первое слово из заголовка транслитом.

In [79]:
# 4. Парсер текстов статей

############################ VARS ################################
TITLES_FILE = "./results/FMEtitles-merged.xml"
PAGES_DIR = "./matphys/rpages/"
EXIT_DIR = "./results/FMEarticles/"
COMBINATIONS_CORR = {
	'І' : 'I'		# This teo are different!
}
##################################################################


class Article:
	start_file = ''
	start_pos = 0
	end_file = ''
	end_pos = 0
	text = ''
	uri = ''
	title = ''
	xml = ''

	def get_text(self):
		if self.start_file == self.end_file:
			with codecs.open(PAGES_DIR + self.start_file, 'r', 'utf-8') as f:
				self.text += f.read()[self.start_pos:self.end_pos]
		else:
			with codecs.open(PAGES_DIR + self.start_file, 'r', 'utf-8') as f:
				self.text += f.read()[self.start_pos:]
			self.text += ' ' # Add a space to prevent word merging
			with codecs.open(PAGES_DIR + self.end_file, 'r', 'utf-8') as f:
				self.text += f.read()[:self.end_pos]
		for comb in COMBINATIONS_CORR.keys():
			while self.text.find(comb) != -1:
				self.text = self.text[:self.text.find(comb)] + COMBINATIONS_CORR[comb] + self.text[self.text.find(comb) + len(comb):]
	
	def make_xml(self):
		self.get_text()

		article = ET.Element("article", {'uri':self.uri})
		title = ET.SubElement(article, 'title')
		title.text = self.title
		author = ET.SubElement(article, 'authors')
		title_short = ET.SubElement(article, 'title_short')
		pages = ET.SubElement(article, 'pages')
		start = ET.SubElement(pages, 'start')
		start.text = self.start_file[3:self.start_file.find('_', 3)]
		end = ET.SubElement(pages, 'end')
		end.text = self.end_file[3:self.end_file.find('_', 3)]
		literature = ET.SubElement(article, 'literature')
		literature_orig = ET.SubElement(literature, 'literature_orig')
		formulas_remote = ET.SubElement(article, 'formulas_main')
		formulas_inline = ET.SubElement(article, 'formulas_aux')
		relations = ET.SubElement(article, 'relations')
		text = ET.SubElement(article, 'text')
		text.text = self.text
		text_orig = ET.SubElement(article, 'text_orig')
		text_orig.text = self.text

		self.xml = prettify(article)
	
	

class Title:
	text = ''
	file = ''
	start_pos = 0
	end_pos = 0
	uri = ''


def get_title(n, root):
	otitle = Title()
	for title in root:
		if int(title.attrib['uri'][30:title.attrib['uri'].find('_', 30)]) == n:
			otitle.uri = title.attrib['uri']
			otitle.text = get_xml_elem(title, 'title').text
			otitle.file = get_xml_elem(title, 'title-meta/title-file').text
			otitle.start_pos = int(get_xml_elem(title, 'title-meta/title-start').text)
			otitle.end_pos = int(get_xml_elem(title, 'title-meta/title-end').text)
	return otitle


root = parse_xml(TITLES_FILE)

# Create articles list
articles_list = []
title = Title()
for i in range(len(root)):
	title = get_title(i + 1, root)
	if i:
		articles_list[-1].end_file = title.file
		articles_list[-1].end_pos = title.start_pos - 2 # There is a shift for some reason
	articles_list.append(Article())
	articles_list[-1].uri = title.uri
	articles_list[-1].title = title.text
	articles_list[-1].start_file = title.file
	articles_list[-1].start_pos = title.end_pos
	articles_list[-1].end_file = title.file
	with codecs.open(PAGES_DIR + title.file, 'r', 'utf-8') as f:
		articles_list[-1].end_pos = len(f.read())

# Parse texts themselves and write
for i in range(len(articles_list)):
	articles_list[i].make_xml()
	with codecs.open(EXIT_DIR + '' + articles_list[i].uri[30:] + '.xml', 'w', 'utf-8') as f:
		f.write(articles_list[i].xml)

# 5. Парсинг авторов статьи

Ищет в конце текста статей конструкции типа ` [Xxxx]. [Xxxx]. [Xxxx]` или ` [Xxxx].[Xxxx]. [Xxxx]` и итерпретирует её как автора статьи.

In [80]:
# 5. Парсинг авторов статьи

############################ VARS ################################
ARTICLES_DIR = "./results/FMEarticles/"
MIN_INLINE_LEN = 0
COMBINATIONS_CORR = dict_merge(COMBINATIONS_CORR_GLOBAL, {
	'II' : 'П'
})
##################################################################


# Get filenames needed
filenames = next(walk(ARTICLES_DIR), (None, None, []))[2]  # [] if no file

for filename in filenames:
	article = parse_xml(ARTICLES_DIR + filename)
	textelem = get_xml_elem(article, 'text')
	text = textelem.text
	authors = get_xml_elem(article, 'authors')

	auth_start = 1
	auth_list = []
	while auth_start:
		# Find first non-space from the end
		while text[-1] == ' ' or text[-1] == '\n' or text[-1] == '\r':
			text = text[:-1]

		auth_start = 0
		# Try recognize
		first_space = max(text.rfind(' ', 0, len(text)), text.rfind('\n', 0, len(text)), text.rfind('\r', 0, len(text)))
		second_space = max(text.rfind(' ', 0, first_space), text.rfind('\n', 0, first_space), text.rfind('\r', 0, first_space))
		third_space = max(text.rfind(' ', 0, second_space), text.rfind('\n', 0, second_space), text.rfind('\r', 0, second_space))
		if first_space >= 0 and text[first_space-1] == '.' and second_space >= 0:
			if text.find('.', second_space, first_space-1) != -1: # If there's no space between initials
				third_space = second_space
				second_space = first_space
			if text[second_space-1] == '.' and third_space >= 0:
				# Check if first letters of each word are capitals
				if re.match(r"[A-ZА-ЯІ]", text[first_space+1]) != None and re.match(r"[A-ZА-ЯІ]", text[second_space+1]) != None and re.match(r"[A-ZА-ЯІ]", text[third_space+1]) != None:
					auth_start = third_space + 1

		if auth_start: # Suggest that an article cannot consist of author only and therefore auth_start should be > 0
			#print(article.attrib['uri'], author_text)
			author_text = text[auth_start:]
			if author_text[author_text.find('.')+1] != ' ': # Add space if there's no one between initials
				author_text = author_text[:author_text.find('.')+1] + ' ' + author_text[author_text.find('.')+1:]
			if author_text[-1] == '.' or author_text[-1] == ',':
				author_text = author_text[:-1]
			# convert wrong symbols
			for comb in COMBINATIONS_CORR.keys():
				while author_text.find(comb) != -1:
					author_text = author_text[:author_text.find(comb)] + COMBINATIONS_CORR[comb] + author_text[author_text.find(comb) + len(comb):]
			
			auth_list.append(author_text)
			text = text[:auth_start]

	# add authors, reverse their order to alphabetic
	for auth in reversed(auth_list):
		author = ET.SubElement(authors, 'author')
		author.text = auth

	textelem.text = text
	with codecs.open(ARTICLES_DIR + filename, 'w', 'utf-8') as f:
		f.write(prettify(article))

# 6. Парсер формул

Выносит из текстов ранее подготовленных xml-файлов статей сначала выносные, а затем строчные формулы, оставляя на их месте ссылку внутри их математического окружения. 

Минимальная длина в символах, которой должна обладать строчная формула, настраивается.

In [81]:
# 6. Парсер формул

############################ VARS ################################
ARTICLES_DIR = "./results/FMEarticles/"
MIN_INLINE_LEN = 0
##################################################################


def check_in_uri(text:str, pos:int) -> bool:
	open_prev = text.rfind('URI[[', 0, pos)
	close_prev = text.rfind(']]/URI', 0, pos)
	open_next = text.find('URI[[', pos)
	close_next = text.find(']]/URI', pos)
	after_open = True if ((open_prev != -1 and close_prev == -1) or (open_prev > close_prev and open_prev != -1 and close_prev != -1)) else False
	before_close = True if ((open_next == -1 and close_next != -1) or (open_next > close_next and open_next != -1 and close_next != -1)) else False
	return (after_open and before_close)



# Get filenames needed
filenames = next(walk(ARTICLES_DIR), (None, None, []))[2]  # [] if no file

for filename in filenames:
	article = parse_xml(ARTICLES_DIR + filename)
	#print('REMOTES: ' + article.attrib['uri'])
	text = get_xml_elem(article, 'text')
	formulas_main = get_xml_elem(article, 'formulas_main')
	formulas_aux = get_xml_elem(article, 'formulas_aux')
			
# Get main formulas
	pos_find = 0
	pos_start = 0
	pos_end = 0
	n = 1
	while text.text != None and text.text.find('\\[', pos_find) != -1:
		pos_start = text.text.find('\\[', pos_find) + 2
		pos_end = text.text.find('\\]', pos_start)
		while text.text[pos_start] == '\n':
			pos_start += 1
		while text.text[pos_end-1] == '\n':
			pos_end -= 1
		pos_find = pos_start
		uri = 'http://libmeta.ru/fme/formula/main' + article.attrib['uri'][article.attrib['uri'].rfind('/', 0, article.attrib['uri'].find('_')):article.attrib['uri'].find('_')+1] + str(n) + article.attrib['uri'][article.attrib['uri'].find('_'):]
		n += 1
		formula = ET.SubElement(formulas_main, 'formula', {'uri':uri})
		formula.text = text.text[pos_start:pos_end]
		text.text = text.text[:pos_start] + 'URI[[' + uri + ']]/URI' + text.text[pos_end:]

# Get auxilary formulas
	pos_find = 0
	pos_start = 0
	pos_end = 0
	cnt = 0
	n = 1
	# Count dollar symbols
	while text.text != None and text.text.find('$', pos_find) != -1:
		pos_find = text.text.find('$', pos_find) + 1
		cnt += 1
	# If cnt is not even assume that first one is garbage from title
	pos_find = 0
	if cnt % 2:
		pos_find = text.text.find('$', pos_find)
		text.text = text.text[:pos_find] + '#' + text.text[pos_find+1:]
	while text.text != None and text.text.find('$', pos_find) != -1:
		pos_start = text.text.find('$', pos_find) + 1
		pos_end = text.text.find('$', pos_start)
		if not check_in_uri(text.text, pos_start) and not check_in_uri(text.text, pos_end):
			while text.text[pos_start] == '\n':
				pos_start += 1
			while text.text[pos_end-1] == '\n':
				pos_end -= 1
			pos_find = pos_start
			if pos_end - pos_start >= MIN_INLINE_LEN:
				uri = 'http://libmeta.ru/fme/formula/aux' + article.attrib['uri'][article.attrib['uri'].rfind('/', 0, article.attrib['uri'].find('_')):article.attrib['uri'].find('_')+1] + str(n) + article.attrib['uri'][article.attrib['uri'].find('_'):]
				n += 1
				formula = ET.SubElement(formulas_aux, 'formula', {'uri':uri})
				formula.text = text.text[pos_start:pos_end]
				text.text = text.text[:pos_start] + 'URI[[' + uri + ']]/URI' + text.text[pos_end:]
			pos_find = text.text.find('$', pos_find) + 1
		else:
			pos_find = pos_end + 1

	with codecs.open(ARTICLES_DIR + filename, 'w', 'utf-8') as f:
		f.write(prettify(article))

## 6.1. Вынос формул

Выносит все формулы в отдельный файл с указанием типа для возможной последующей обработки.

In [ ]:
# 6.1. Вынос формул

############################ VARS ################################
ARTICLES_DIR = "./results/FMEarticles/"
EXIT_FILE = "./results/FMEformulas.xml"
##################################################################


# Get filenames needed
filenames = next(walk(ARTICLES_DIR), (None, None, []))[2]  # [] if no file


formulas = ET.Element('formulas')

for filename in filenames:
	root = parse_xml(ARTICLES_DIR + filename)
	fmain = get_xml_elem(root, 'formulas_main')
	faux = get_xml_elem(root, 'formulas_aux')
	
	for formula in fmain:
		formulas.append(formula)
	for formula in faux:
		formulas.append(formula)

with codecs.open(EXIT_FILE, 'w', 'utf-8') as f:
	f.write(prettify(formulas))

## 6.2. Проверка формул

Случайным образом выбирает 20 случайных формул (из случайных статей) и ставляет их в математическое окружение Markdown для визуальной проверки

In [ ]:
# 6.2. Проверка формул

############################ VARS ################################
ARTICLES_DIR = "./results/FMEarticles/"
EXIT_FILE = "./matphys/FMEformulas_check.md"
NUMBER = 20
##################################################################


# Get filenames needed
filenames = next(walk(ARTICLES_DIR), (None, None, []))[2]  # [] if no file


file = ''

i = 0
while i < NUMBER:
	root = parse_xml(ARTICLES_DIR + filenames[randint(0, len(filenames)-1)])

	# Get all the info from article
	fmain = get_xml_elem(root, 'formulas_main')
	start = get_xml_elem(root, 'pages/start').text
	

	# if there's no formulas in the article try another one
	total_num = 0
	for formula in fmain:
		total_num += 1
	if not total_num:
		continue
	i += 1
	
	num = randint(0, 100) % total_num

	formula = fmain[num].text

	file += f'{i}. Статья: {root.attrib["uri"]}, Начало на стр. {start}, формула {num + 1}:\n$${formula}$$\n'

with codecs.open(EXIT_FILE, 'w', 'utf-8') as f:
	f.write(file)